In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import rc # 폰트
from datetime import datetime
import time
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
# datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')

pd.set_option ('display.max_rows', 10)



path = 'C:\\Users\\wldbs\\Desktop\\kart\\1031\\'

In [25]:
crawl = pd.read_csv(path + '00-crawling-track.csv')

In [7]:
# merge를 위해 변수명 변경

crawl.columns = ['map_name', '태그', '난이도', '랩', '트랙 길이', '진행 방향', '트랙 분류', 'AI 주행', '첫 등장일','리그 트랙', '라이센스', '약칭']

In [8]:
# 차이나 북경 경기장의 경우 난이도, 랩 잘못 입력되어있음. 아예 삭제하는 게 낫겠다는 판단
drop_data = crawl[crawl['난이도'].isnull()].index

crawl.drop(drop_data, inplace = True)

In [9]:
# 결측치 대치 (missing 이라는 문자값으로 입력되게 함)

crawl = crawl.fillna('missing')

In [10]:
# 필요 없는 컬럼 제거

crawl = crawl.drop(['태그', '트랙 분류', '리그 트랙', '라이센스', '약칭', 'AI 주행'], axis = 1)

In [11]:
# 난이도 문자열 - 숫자열로 변경

crawl['난이도'] = crawl.난이도.str.split('[').str[0]
crawl['난이도'] = crawl.난이도.str.split(' ').str[0]

crawl.난이도.unique()

crawl['난이도'].replace( {'●○○○○' : 1,
                          '●●○○○' : 2,
                          '●●●○○' : 3,
                          '●●●●○' : 4,
                          '●●●●●' : 5,
                          '●●○○○○' : 2,
                          '●●●●●○' : 4,
                          '●●●○○○' : 3,
                          '●●●●●●' : 5
                          }, inplace = True)

In [12]:
# 랩 문자열 - 숫자열로 변경

crawl['랩'] = crawl.랩.str.split('바').str[0]
crawl['랩']= crawl.랩.astype('int')


# 트랙 길이 문자열 - 숫자열로 변경

crawl['트랙 길이'].unique()

crawl['트랙 길이'] = crawl['트랙 길이'].str.split('k').str[0]
crawl['트랙 길이'] = crawl['트랙 길이'].str.split(' ').str[0]
crawl['트랙 길이'].replace( {'???' : 0 }, inplace = True)

crawl['트랙 길이'] = crawl['트랙 길이'].astype('float')


# 범주형 변수 따로 변환하지 않기로 함
crawl['진행 방향'].unique()


# # 날짜형 변환

crawl['년'] = crawl['첫 등장일'].str.split(' ').str[0]
crawl['월'] = crawl['첫 등장일'].str.split(' ').str[1]
crawl['일'] = crawl['첫 등장일'].str.split(' ').str[2]

crawl['년'] = crawl['년'].str.strip('년').str[0:4]
crawl['월'] = crawl['월'].str.split('월').str[0]
crawl['일'] = crawl['일'].str.split('일').str[0]

crawl['월'] = crawl['월'].str.rjust(width=2,fillchar='0')        # rlpad
crawl['일'] = crawl['일'].str.rjust(width=2,fillchar='0')        # rlpad


# 년 변환
crawl['년'].replace( {'200-' : np.NaN,
                      'miss' : np.NaN,
                      '-'    : np.NaN}, inplace = True)

# 월 변환
crawl['월'].replace( {'0-' : np.NaN,
                      '0?' : np.NaN
                      }, inplace = True)

# 일 변환
crawl['일'].replace( {'0-' : np.NaN,
                      '0?' : np.NaN
                      }, inplace = True)


# 출시일 컬럼 생성

crawl['출시일'] = crawl['년'] + crawl['월'] + crawl['일']


# 날짜형으로 변경

crawl['출시일'] = pd.to_datetime(crawl['출시일'])


# 출시일의 nan 값 0으로 변경

crawl = crawl.fillna('0')


# 중복 데이터 있는지 확인
# 30개 행의 map_name이 중복되고 있음

crawl.map_name.nunique()


# 중복 제거
crawl = crawl.drop_duplicates()

In [19]:
crawl

,map_name,난이도,랩,트랙 길이,진행 방향,첫 등장일,년,월,일,출시일
0,포레스트 통나무,1,3,6.0,반시계방향,2004년 6월 1일,2004,06,01,2004-06-01 00:00:00
1,포레스트 버섯동굴,2,2,5.2,시계방향,2004년 7월 20일,2004,07,20,2004-07-20 00:00:00
2,포레스트 골짜기,1,3,7.5,시계방향,2004년 6월 7일,2004,06,07,2004-06-07 00:00:00
3,포레스트 기암괴석,2,3,5.4,시계방향,2004년 6월 29일,2004,06,29,2004-06-29 00:00:00
4,포레스트 폭포속으로,2,3,4.4,시계방향,2006년 4월 20일,2006,04,20,2006-04-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...
330,코리아 부산의 밤,3,2,8.2,반시계방향,2021년 7월 22일,2021,07,22,2021-07-22 00:00:00
331,코리아 제주 해오름 다운힐,2,1,5.6,단선형,2021년 8월 12일,2021,08,12,2021-08-12 00:00:00
332,[R] 코리아 다이나믹 서울,3,1,6.6,반시계방향,2022년 2월 17일,2022,02,17,2022-02-17 00:00:00
333,[R] 코리아 부산의 밤,3,2,8.2,시계방향,2022년 2월 17일,2022,02,17,2022-02-17 00:00:00


In [18]:
map_name_with_theme

,테마,trackId,map_name
0,빌리지,c93843288b6c038f11328c0e4a4dae0663a94154242059...,빌리지 두개의 관문
1,팩토리,c5d2203fc4979dab295e07763de827776f4e83c373cc66...,팩토리 핀저의 위험한 적재소
2,올림포스,bfd558c15104ec3ef0f182893a3c9bd77052ce7675028c...,올림포스 선택의 문
3,WKC,e86138a3a04c138f991da883d4052ff3a00624ba9eb29d...,WKC 싱가폴 마리나 서킷
4,어비스,bced4f65cf36c22df94c6e26436e37f22b1cf42b5878e2...,어비스 스카이라인
...,...,...,...
338,팩토리,ee9f7fc6ac66066247bc8de759996057ae42a2a5d2283f...,팩토리 브로디의 심술
339,문힐시티,6f39b6766c8a93b662250c86c038154139b42448d12f0c...,문힐시티 지피의 미완성 빌딩
340,월드,783f08e9fdaa169ec3138eecf72451e9439211d414e200...,월드 그리스의 휴일
341,해변,3b3d58a41efb9b00da3e88874a90b2e97ce0bc43381e7a...,비치 신나는 여름 휴가


In [14]:
map_name_with_theme = pd.read_csv(path + '00-map-name-with-theme.csv', encoding = 'utf-8')

In [21]:
map_name_with_theme.isnull().sum()

테마          13
trackId      0
map_name     0
dtype: int64

### right join 하기

In [15]:
merged_crwal = pd.merge(crawl, map_name_with_theme, on = 'map_name', how = 'right')

In [20]:
# 저장

merged_crwal.to_csv(path + '00_crawl_data_with_theme.csv', index=False, encoding = 'utf-8')